In [184]:
import pandas as pd

# Path to CSV file
spreadspoke_scores_file = "../CSV_Files/spreadspoke_scores.csv"
nfl_teams_file = "../CSV_Files/nfl_teams.csv"
# Read CSV file into DataFrame
spreadspoke_scores_df = pd.read_csv(spreadspoke_scores_file)
nfl_teams_df = pd.read_csv(nfl_teams_file)

# Data inspection
# spreadspoke_scores_df

In [185]:
# Data cleaning and filtering
nfl_teams_df['Abbreviation'] = nfl_teams_df['Abbreviation'].replace('LV', 'LVR')
last_5_years_df = spreadspoke_scores_df[spreadspoke_scores_df['schedule_season'] >= 2019]
clean_df = last_5_years_df.drop(columns=['stadium', 'stadium_neutral', 'weather_temperature', 'weather_wind_mph',
                              'weather_humidity', 'weather_detail', 'spread_favorite', 'over_under_line'])
# clean_df


In [186]:
# Count of favorite team appearances
team_favorite_count_df = pd.DataFrame(columns=['Abbreviation', 'Favorite Pick'])
for team_abbreviation in nfl_teams_df['Abbreviation']:
    count = clean_df[clean_df['team_favorite_id'] == team_abbreviation].shape[0]
    team_favorite_count_df = pd.concat([team_favorite_count_df, pd.DataFrame({'Abbreviation': [team_abbreviation], 'Favorite Pick': [count]})])

# Sort teams by favorite count from highest to lowest
team_favorite_count_df = team_favorite_count_df.sort_values(by='Favorite Pick', ascending=False)
# team_favorite_count_df

In [187]:
# Count of favorite team appearances

favorite_wins = clean_df[clean_df["score_home"] > clean_df["score_away"]]["team_favorite_id"].value_counts()
favorite_wins_df = favorite_wins.reset_index()
favorite_wins_df.columns = ['Abbreviation', 'Win Count']
# favorite_wins_df

In [189]:
# Merge Dateframes
merged_f_t = pd.merge(favorite_wins_df, team_favorite_count_df, on='Abbreviation')
merged_df = pd.merge(merged_f_t, nfl_teams_df, on='Abbreviation')
merged_df = merged_df.reindex(['Name', 'Abbreviation', 'Win Count', 'Favorite Pick', 'Win Rate'], axis=1)

# Calculate win percentage for each team
for row in merged_df:
    ft_win = merged_df['Win Count'].astype(int)
    ft_pick = merged_df['Favorite Pick'].astype(int)
    win_rate = round(ft_win/ft_pick * 100, 2)

# Add Win rate into merge_df dataframe
win_rate_df = merged_df
win_rate_df['Win Rate'] = pd.DataFrame({'Win Rate' : win_rate})
win_rate_df

,Name,Abbreviation,Win Count,Favorite Pick,Win Rate
0,Kansas City Chiefs,KC,48,87,55.17
1,Buffalo Bills,BUF,41,69,59.42
2,Green Bay Packers,GB,38,59,64.41
3,San Francisco 49ers,SF,37,73,50.68
4,Dallas Cowboys,DAL,36,62,58.06
5,Philadelphia Eagles,PHI,34,58,58.62
6,Baltimore Ravens,BAL,33,66,50.00
7,Cleveland Browns,CLE,32,49,65.31
8,Tampa Bay Buccaneers,TB,29,60,48.33
9,Los Angeles Rams,LAR,29,55,52.73


In [191]:
# Moneyline 베팅 값을 계산하는 함수
def calculate_moneyline(win_rate):
    return 100 / win_rate

# Moneyline 베팅 값을 계산하여 새로운 열에 추가
win_rate_df['Moneyline'] = win_rate_df['Win Rate'].apply(calculate_moneyline)

# 결과 출력
win_rate_df

,Name,Abbreviation,Win Count,Favorite Pick,Win Rate,Moneyline
0,Kansas City Chiefs,KC,48,87,55.17,1.812579
1,Buffalo Bills,BUF,41,69,59.42,1.682935
2,Green Bay Packers,GB,38,59,64.41,1.552554
3,San Francisco 49ers,SF,37,73,50.68,1.973165
4,Dallas Cowboys,DAL,36,62,58.06,1.722356
5,Philadelphia Eagles,PHI,34,58,58.62,1.705902
6,Baltimore Ravens,BAL,33,66,50.00,2.000000
7,Cleveland Browns,CLE,32,49,65.31,1.531159
8,Tampa Bay Buccaneers,TB,29,60,48.33,2.069108
9,Los Angeles Rams,LAR,29,55,52.73,1.896454
